# Tutorial
- Syndp 패키지를 이용해서 데이터를 합성하는 알고리즘에 대한 튜토리얼입니다.
- input 데이터가 어떻게 들어가는지, output은 어떻게 생겼는지 보여드리고,
- code 파일에 있는 알고리즘에 대한 설명으로 튜토리얼을 마무리 하도록 하겠습니다.



## Data expained
- 데이터는 data 폴더에 들어있음.
- 데이터는 MIMIC 데이터의 일부 시계열 데이터를 가져왔음.
```
data━┳━ processed*

```

- 데이터는 다변량 형태이나 단일 변량으로 processing 하여 syndp에서 사용할 수 있도록 만들어주겠습니다.

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

project_dir = Path().cwd().parent
data_dir = project_dir.joinpath('data')

# load data
death = pd.read_csv(data_dir.joinpath('processed/death_df.csv'))

death.head()

,subject_id,stay_id,hour,heart_rate,sbp,dbp,mbp,resp_rate,spo2
0,10001884,37510196,0,68.0,138.0,77.0,96.0,19.0,99.0
1,10001884,37510196,1,65.0,121.0,73.0,87.0,19.0,97.0
2,10001884,37510196,2,63.0,123.0,72.0,86.0,18.0,97.0
3,10001884,37510196,3,63.0,125.0,74.0,90.0,20.0,97.0
4,10001884,37510196,4,66.0,107.0,67.0,79.0,19.0,97.0


위에서 보이듯 tabular data 형태이며 hour로 시간에 대한 index가 나타나 있으며 hr, sbp, mbp 등 시계열 연속형 변수를 볼 수 있다.

본 tutorial에서는 단일 변량에 대해서 검증하는 방법을 숙지하는 것을 목표로 하겠음. 여기서는 HR, hour, ID 정보만 사용한다.

In [10]:
sample = death[['subject_id','hour','heart_rate']].copy()

In [16]:
# fillna 선형보간법으로 linear interpolation 해준다.

sample.interpolate()

,subject_id,hour,heart_rate
0,10001884,0,68.0
1,10001884,1,65.0
2,10001884,2,63.0
3,10001884,3,63.0
4,10001884,4,66.0
...,...,...,...
106117,19999840,19,60.0
106118,19999840,20,61.0
106119,19999840,21,68.5
106120,19999840,22,76.0
